In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymap3d as pm
%matplotlib widget

In [2]:
def transform(t_x, t_y, t_z, r_x, r_y, r_z, r_w):
    return np.asarray(
        [[1-2*r_y**2-2*r_z**2, 2*r_x*r_y-2*r_z*r_w, 2*r_x*r_z+2*r_y*r_w, t_x],
         [2*r_x*r_y+2*r_z*r_w, 1-2*r_x**2-2*r_z**2, 2*r_y*r_z-2*r_x*r_w, t_y],
         [2*r_x*r_z-2*r_y*r_w, 2*r_y*r_z+2*r_x*r_w, 1-2*r_x**2-2*r_y**2, t_z],
         [0,0,0,1]])
# https://en.wikipedia.org/wiki/Rotation_matrix#General_rotations

In [3]:
df = pd.read_csv("/home/torsten/ms/results/citrus/04fg.csv", header=0, skiprows=[])
df

,correction_count,key_index,t_x,t_y,t_z,v_x,v_y,v_z,r_x,r_y,r_z,r_w,sec,nanosec
0,1,0,-2.322160e-12,-7.459180e-13,1.069460e-13,-0.227384,-0.073403,0.008752,0.003420,-0.010595,-6.688220e-09,0.999938,1689715534,288268504
1,1,1,3.065850e-02,9.491330e-03,-1.827260e-04,0.256560,0.074877,-0.008718,0.004841,-0.011265,-1.229000e-03,0.999924,1689715536,88136504
2,2,0,-2.297980e-11,-6.535970e-12,1.424070e-12,-0.053042,-0.031081,0.013225,0.005732,-0.020107,-2.761250e-09,0.999781,1689715534,288268504
3,2,1,-1.374420e-02,-1.321530e-03,1.456670e-02,0.013233,0.015814,0.002307,0.007997,-0.024196,-1.209250e-03,0.999675,1689715536,88136504
4,2,2,2.325100e-01,5.083620e-03,1.445680e-02,0.659213,-0.015153,0.019249,0.008096,-0.025810,9.609110e-04,0.999634,1689715538,588150504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47580,307,303,-2.106700e+00,-5.459000e-01,9.694290e+00,-0.677850,0.344670,0.018497,0.000502,-0.008471,9.645750e-01,0.263674,1689716291,87270504
47581,307,304,-2.955430e+00,1.457020e+00,9.720630e+00,0.154914,0.682079,0.037926,-0.004408,-0.012379,5.638310e-01,0.825785,1689716293,587431504
47582,307,305,-1.417410e+00,2.160360e+00,9.768360e+00,1.215780,0.075955,-0.033351,0.004182,-0.014240,-2.934150e-03,0.999886,1689716296,87310504
47583,307,306,7.397340e-01,2.534800e+00,9.833520e+00,0.149703,-0.013815,0.015760,0.001843,-0.007871,8.309820e-02,0.996509,1689716298,587310504


In [4]:
# -33.7892578,18.9620374,150.365

#origin_tup = (-33.7888318,18.9602676,152.754)# yard 22
#origin_tup = (-33.7892517,18.9620424,149.669)# reeds 
#origin_tup = (-33.7892578,18.9620374,150.365)# trig 
#origin_tup = (-33.7888752,18.9605481,153.477)# building
origin_tup = (-33.7888915,18.9606631,153.051)# yard 5

lat0 = origin_tup[0]
lon0 = origin_tup[1]
h0 = origin_tup[2]

df_gnss = pd.read_csv("/home/torsten/ms/data/farm20240202/rosbag2_2024_02_02-10_06_39/ublox.csv")
df_gnss

,sec,nanosec,lat,lon,alt,vel_n,vel_e,vel_d,h_acc
0,1706868401,363441322,-33.788862,18.960538,153.015,-0.02,-0.01,0.01,1.856
1,1706868401,757919459,-33.788862,18.960538,153.049,0.00,0.00,0.00,1.856
2,1706868402,154177291,-33.788862,18.960539,153.086,-0.01,0.01,0.01,1.857
3,1706868402,561323807,-33.788862,18.960538,153.056,0.05,-0.06,0.03,1.858
4,1706868402,954264605,-33.788862,18.960539,153.062,0.03,0.04,0.03,1.858
...,...,...,...,...,...,...,...,...,...
995,1706868802,574390147,-33.789150,18.961348,153.292,-0.02,0.00,0.00,1.902
996,1706868802,964066973,-33.789150,18.961348,153.295,-0.01,0.01,0.01,1.903
997,1706868803,365576020,-33.789150,18.961348,153.283,0.01,0.01,0.02,1.905
998,1706868803,789906964,-33.789150,18.961348,153.278,0.00,-0.01,0.00,1.906


In [5]:
# Sat image segments
segment_filenames = ["1.csv", "2.csv", "3.csv", "4.csv"]
segment_coords = []
segments_enu = []

for fn in segment_filenames:
    df_seg = pd.read_csv("~/ms/data/old_zoo_sat_paths/" + fn)
    segment_coords.append(df_seg.iloc[:, 0:2].values)

for seg in segment_coords:
    segments_enu.append(np.zeros(seg.shape))
    for i in range(seg.shape[0]):
        enu = pm.geodetic2enu(seg[i, 1], seg[i, 0], 102, lat0, lon0, h0)
        segments_enu[-1][i, 0] = enu[0]
        segments_enu[-1][i, 1] = enu[1]
    

In [6]:

start = 0
%matplotlib tk
plt.figure(figsize=(12, 8))
plt.title("Factor Graph")
plt.axis('equal')
plt.grid()

num_sets = df.iloc[-1,0]

# Factor graph
for i in range(0, num_sets, 1):
    plt.plot(df.iloc[start:start+i+2, 2], df.iloc[start:start+i+2, 3], "-x")#, color="grey")
    plt.text(df.iloc[start+i+1, 2], df.iloc[start+i+1, 3], str(i+1))
    
    start+=i+2

# GNSS trail
"""
for index, row in df_gnss.iterrows():
    if index % 10 != 0:
        continue
    enu = pm.geodetic2enu(row["lat"], row["lon"], row["alt"], lat0, lon0, h0)
    plt.plot(enu[0], enu[1], "-bx")
    plt.errorbar(enu[0], enu[1], row["h_acc"], row["h_acc"])
    

# Sat image segments
for seg in segments_enu:
    #plt.plot(seg[:, 0], seg[:, 1], "-", color="blue")
    pass
"""
plt.show()

In [7]:

start = 0

plt.figure()

plt.subplot(2,1,1);
plt.title("Z - height")

num_sets = df.iloc[-1,0]

# Factor graph
for i in range(0, num_sets, 1):
    plt.plot(df.iloc[start:start+i+2, 4])
    #plt.text(df.iloc[start+i+1, 2], df.iloc[start+i+1, 3], str(i+1))
    
    start+=i+2

plt.subplot(2,1,2);
plt.title("Speed m/s")
start = 0
for i in range(0, num_sets, 1):
    plt.plot(np.sqrt(df.iloc[start:start+i+2, 5]**2 + df.iloc[start:start+i+2, 6]**2 + df.iloc[start:start+i+2, 7]**2))
    #plt.text(df.iloc[start+i+1, 2], df.iloc[start+i+1, 3], str(i+1))
    
    start+=i+2

# GNSS trail
"""
for index, row in df_gnss.iterrows():
    if index % 10 != 0:
        continue
    enu = pm.geodetic2enu(row["lat"], row["lon"], row["alt"], lat0, lon0, h0)
    plt.plot(enu[0], enu[1], "-bx")
    plt.errorbar(enu[0], enu[1], row["h_acc"], row["h_acc"])
    """

# Sat image segments
for seg in segments_enu:
    #plt.plot(seg[:, 0], seg[:, 1], "-", color="blue")
    pass

plt.show()

In [8]:
#Online estimate
index = -1
for i in range(num_sets):
    #plt.plot(df.iloc[start:start+i+2, 2], df.iloc[start:start+i+2, 3], "-x", color="grey")
    #plt.text(df.iloc[start+i+1, 2], df.iloc[start+i+1, 3], str(i+1))
    index += (i + 2)
    row = df.iloc[index,:]
    wgs = pm.enu2geodetic(row[2], row[3], row[4], lat0, lon0, h0)
    print(wgs[1], wgs[0], 0, sep=",", end=" ")

18.960663431031982,-33.788891414431816,0 18.960665610502666,-33.78889145416908,0 18.960680620611157,-33.78889120710087,0 18.960686319584397,-33.788891067291736,0 18.960710723415882,-33.78889224529774,0 18.96074521903484,-33.788892674012445,0 18.960779049968902,-33.78889216135691,0 18.9608127259494,-33.7888894691539,0 18.960846076933336,-33.7888912745451,0 18.960878958227656,-33.788890749345725,0 18.960911899975336,-33.788886698047186,0 18.96094401682051,-33.78888833520221,0 18.960977689574946,-33.78888927338079,0 18.961011319146046,-33.78888989377542,0 18.96104532877929,-33.78889064479386,0 18.96107787535982,-33.78889113416376,0 18.961111706821036,-33.78888933427998,0 18.961145302916368,-33.78889006524826,0 18.961179516616017,-33.78889038997981,0 18.96121348307568,-33.78889181679351,0 18.961245260921885,-33.78889874414297,0 18.961258147802464,-33.78892406841867,0 18.961261284648494,-33.78895179924064,0 18.96125917610788,-33.78897955812516,0 18.96125872604944,-33.78900658468169,0 18.961

/tmp/ipykernel_203600/1247323099.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  wgs = pm.enu2geodetic(row[2], row[3], row[4], lat0, lon0, h0)


In [9]:
#Final estimate
index = int(0.5*num_sets*(num_sets+1)-1)
for i in range(num_sets):
    #plt.plot(df.iloc[start:start+i+2, 2], df.iloc[start:start+i+2, 3], "-x", color="grey")
    #plt.text(df.iloc[start+i+1, 2], df.iloc[start+i+1, 3], str(i+1))
    
    row = df.iloc[index+i,:]
    wgs = pm.enu2geodetic(row[2], row[3], row[4], lat0, lon0, h0)
    print(wgs[1], wgs[0], 0, sep=",", end=" ")

/tmp/ipykernel_203600/1253511891.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  wgs = pm.enu2geodetic(row[2], row[3], row[4], lat0, lon0, h0)


18.9606631,-33.788891500000005,0 18.9606627348083,-33.788891566200036,0 18.960665835509527,-33.78889145506259,0 18.960681250638395,-33.78889123719332,0 18.960686088842525,-33.78889102673602,0 18.960712167464667,-33.788891795569256,0 18.96074570902795,-33.788892024132394,0 18.96077935985684,-33.788890905632755,0 18.960812803693155,-33.78888762150109,0 18.96084619678722,-33.78888871805362,0 18.960879102913594,-33.78888743025847,0 18.960911841668377,-33.78888256852426,0 18.96094404272906,-33.78888341934689,0 18.960977769460072,-33.788883277651564,0 18.961011388217592,-33.78888280867018,0 18.96104522832278,-33.78888247345847,0 18.96107789151064,-33.788881785658084,0 18.961111546964606,-33.788878618339496,0 18.961145143045357,-33.78887794575394,0 18.96117939560683,-33.78887696293645,0 18.96121325622427,-33.78887660740294,0 18.96124573266389,-33.78888279950971,0 18.96126008688873,-33.78890669572907,0 18.961265640192035,-33.78893482411777,0 18.961265182584967,-33.788963069646066,0 18.96126719

# Comparing position and orientation to GNSS trail

- Get path with points and timestamps
- Iterate over optimisation points of path
- Find next closest GNSS point to timestamp
- Compare location, angles ect
- Record to result array
- Plot errors and some statistical values

## Add Heading / Orientation
- 



In [10]:
# Tested
def degree_heading_velNE(vel_N, vel_E):
    if vel_N == 0:
        if vel_E > 0:
            return 90
        elif vel_E < 0:
            return 270
        else:
            return 0
    # opposite / adjacent
    minor_ang = np.arctan(np.abs(vel_E) / np.abs(vel_N))
    # N+, E+
    if vel_N > 0 and vel_E >= 0:
        return np.rad2deg(minor_ang)
    # N-, E+
    elif vel_N < 0 and vel_E >= 0:
        return 180 - np.rad2deg(minor_ang)
    # N-, E-
    elif vel_N < 0 and vel_E < 0:
        return 180 + np.rad2deg(minor_ang)
    # N+, E-
    else:
        return 360 - np.rad2deg(minor_ang)
        

def degree_heading_rot_ENU(rot_ENU):
    vel_E = rot_ENU[0,0]
    vel_N = rot_ENU[1,0]
    return degree_heading_velNE(vel_E, vel_N)

def heading_err(h1, h2):
    err = np.abs(h1 - h2)
    if err > 180:
        return 360 - err
    else:
        return err

In [11]:
#Online estimate
index_fg = -1
index_gnss = 0
row_gnss = None

err_E = []
err_N = []
heading_GNSS = []
heading_fg = [] # Velocity based
orientation_fg = [] # Orientation based

heading_err_v = []

for i in range(num_sets):
    index_fg += (i + 2)
    row_fg = df.iloc[index_fg,:]
    # Get corresponding GNSS point for optimisation point on Factor Graph
    while True:
        row_gnss = df_gnss.iloc[index_gnss,:]
        ts_gnss = row_gnss["sec"] * 1000 + row_gnss["nanosec"] // 1_000_000
        if row_fg["timestamp"] <= ts_gnss + 400:
            break
        index_gnss = index_gnss + 1

    # print (i, row_fg["timestamp"], ts_gnss, row_fg["timestamp"]-ts_gnss)
    enu_gnss = pm.geodetic2enu(row_gnss["lat"], row_gnss["lon"], row_gnss["alt"], lat0, lon0, h0)

    err_E.append(enu_gnss[0] - row_fg["t_x"])
    err_N.append(enu_gnss[1] - row_fg["t_y"])
    heading_GNSS.append(degree_heading_velNE(row_gnss["vel_n"], row_gnss["vel_e"]))
    heading_fg.append(degree_heading_velNE(row_fg["v_x"], row_fg["v_y"]))
    tf = transform(0,0,0,row_fg["r_x"],row_fg["r_y"],row_fg["r_z"],row_fg["r_w"])
    orient = degree_heading_rot_ENU(tf)
    orientation_fg.append(orient)

    heading_err_v.append(heading_err(heading_GNSS[-1], orient))
    

err_E = np.array(err_E)
err_N = np.array(err_N)

err_pos = np.sqrt(err_E**2 + err_N**2)
err_head = np.array(heading_err_v)

print ("Mean position error:", np.mean(err_pos))
print ("Mean heading error:", np.mean(err_head))

plt.figure(figsize=(12,6))

plt.subplot(7,1,1);
plt.title("E error")
plt.plot(err_E)

plt.subplot(7,1,2);
plt.title("N error")
plt.plot(err_N)

plt.subplot(7,1,3);
plt.title("Abs error")
plt.plot(np.sqrt(err_E**2 + err_N**2))

plt.subplot(7,1,4);
plt.title("Heading GNSS")
plt.plot(heading_GNSS)

plt.subplot(7,1,5);
plt.title("Heading FG vel")
plt.plot(heading_fg)

plt.subplot(7,1,6);
plt.title("Orientation FG in ENU Frame")
plt.plot(orientation_fg)

plt.subplot(7,1,7);
plt.title("Heading Error")
plt.plot(heading_err_v)

#plt.show()

    

KeyError: 'timestamp'

In [ ]:
#Final estimate
index_fg = int(0.5*num_sets*(num_sets+1)-1)
index_gnss = 0
row_gnss = None

err_E = []
err_N = []
heading_GNSS = []
heading_fg = [] # Velocity based
orientation_fg = [] # Orientation based

heading_err_v = []

for i in range(num_sets):
    index_fg += 1
    row_fg = df.iloc[index_fg,:]
    # Get corresponding GNSS point for optimisation point on Factor Graph
    while True:
        row_gnss = df_gnss.iloc[index_gnss,:]
        ts_gnss = row_gnss["sec"] * 1000 + row_gnss["nanosec"] // 1_000_000
        if row_fg["timestamp"] <= ts_gnss + 400:
            break
        index_gnss = index_gnss + 1

    # print (i, row_fg["timestamp"], ts_gnss, row_fg["timestamp"]-ts_gnss)
    enu_gnss = pm.geodetic2enu(row_gnss["lat"], row_gnss["lon"], row_gnss["alt"], lat0, lon0, h0)

    err_E.append(enu_gnss[0] - row_fg["t_x"])
    err_N.append(enu_gnss[1] - row_fg["t_y"])
    heading_GNSS.append(degree_heading_velNE(row_gnss["vel_n"], row_gnss["vel_e"]))
    heading_fg.append(degree_heading_velNE(row_fg["v_x"], row_fg["v_y"]))
    tf = transform(0,0,0,row_fg["r_x"],row_fg["r_y"],row_fg["r_z"],row_fg["r_w"])
    orient = degree_heading_rot_ENU(tf)
    orientation_fg.append(orient)

    heading_err_v.append(heading_err(heading_GNSS[-1], orient))
    

err_E = np.array(err_E)
err_N = np.array(err_N)

err_pos = np.sqrt(err_E**2 + err_N**2)
err_head = np.array(heading_err_v)

print ("Mean position error:", np.mean(err_pos))
print ("Mean heading error:", np.mean(err_head))

#plt.figure(figsize=(12,6))

plt.subplot(7,1,1);
plt.title("E error")
plt.plot(err_E)

plt.subplot(7,1,2);
plt.title("N error")
plt.plot(err_N)

plt.subplot(7,1,3);
plt.title("Abs error")
plt.plot(np.sqrt(err_E**2 + err_N**2))

plt.subplot(7,1,4);
plt.title("Heading GNSS")
plt.plot(heading_GNSS)

plt.subplot(7,1,5);
plt.title("Heading FG vel")
plt.plot(heading_fg)

plt.subplot(7,1,6);
plt.title("Orientation FG in ENU Frame")
plt.plot(orientation_fg)

plt.subplot(7,1,7);
plt.title("Heading Error")
plt.plot(heading_err_v)

plt.show()

    